In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client(n_workers=3, threads_per_worker=1) # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants
import datetime

sys.path.append("../..")
import processing_dask as pr
import plot_dask
import processing as old_processing
import px4_log_loader

sys.path.append("../../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
import matplotlib
# matplotlib.rcParams.update({
#         'font.size': 16,
#         'legend.fontsize': 10,
#         'lines.linewidth': 2,
#         "svg.fonttype": 'none'
#     })

In [ ]:
t_start = datetime.datetime.now()

## Load Data

In [ ]:
base_path = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/"
need_postprocessing_phase_inversion = False

# 9-1 Vatnajokull Day 2

# prefix = base_path + "drone/radar_data/20220901-vatnajokull-day2/20220901_095129" # Flight 1 -- possibly too aggressive PRF
# px4_logs = base_path + "px4_logs/2022-09-01/16_40_36.ulg"

# prefix = base_path + "drone/radar_data/20220901-vatnajokull-day2/20220901_105725" # Flight 2
# px4_logs = base_path + "px4_logs/2022-09-01/17_43_54.ulg"

# Vatnajokull Day 4

# prefix = base_path + "drone/radar_data/20220903-vatnajokull-day4/20220903_035332" # Flight 1 - V2 to V1 transect
# px4_logs = base_path + "px4_logs/2022-09-03/10_40_26.ulg"

# prefix = base_path + "drone/radar_data/20220903-vatnajokull-day4/20220903_070453" # Flight 2
# px4_logs = base_path + "px4_logs/2022-09-03/13_49_57.ulg"

# prefix = base_path + "drone/radar_data/20220903-vatnajokull-day4/20220903_091901" # Flight 3
# px4_logs = base_path + "px4_logs/2022-09-03/16_03_58.ulg" # Excluding for lots of clipping

# Vatnajokull Day 5 -- SKF6

# prefix = base_path + "drone/radar_data/20220904-vatnajokull-day5/20220904_041423" # Flight 1.5 (forgot to turn on radar for first one)
# px4_logs = base_path + "px4_logs/2022-09-04/11_04_34.ulg"

# prefix = base_path + "drone/radar_data/20220904-vatnajokull-day5/20220904_054559" # Flight 2 (the long one)
# px4_logs = base_path + "px4_logs/2022-09-04/12_10_47.ulg"

## Slakbreen 2 -- Skipping data from this day to a roll oscillation issue from an unglued servo horn

# prefix = base_path + "drone/radar_data/20230314-slakbreen-day2/20230314_073911" # Flight 1
# px4_logs = base_path + "px4_logs/2023-03-14/14_34_05.ulg"

# prefix = base_path + "drone/radar_data/20230314-slakbreen-day2/20230314_080735" # Flight 2
# px4_logs = base_path + "px4_logs/2023-03-14/14_59_01.ulg"

# Slakbreen 3

# prefix = base_path + "drone/radar_data/20230315-slakbreen-day3/20230315_064228" # Flight 1
# px4_logs = base_path + "px4_logs/2023-03-15/13_35_40.ulg"

# prefix = base_path + "drone/radar_data/20230315-slakbreen-day3/20230315_070941" # Flight 2
# px4_logs = base_path + "px4_logs/2023-03-15/14_03_44.ulg" # Clipping a little, but doesn't seem too bad

# prefix = base_path + "drone/radar_data/20230315-slakbreen-day3/20230315_074655" # Flight 3
# px4_logs = base_path + "px4_logs/2023-03-15/14_38_58.ulg"

# Slakbreen 5

# prefix = base_path + "drone/radar_data/20230317-slakbreen-day5/20230317_033041" # flight 1 (2x liion, 2x wing ext)
# px4_logs = base_path + "px4_logs/2023-03-17/10_20_31.ulg"

# prefix = base_path + "drone/radar_data/20230317-slakbreen-day5/20230317_042115" # flight 2 (2x liion, 1x wing ext)
# px4_logs = base_path + "px4_logs/2023-03-17/10_58_17.ulg"

#prefix = base_path + "drone/radar_data/20230317-slakbreen-day5/20230317_061000" # flight 3 (2x liion, 1x wing ext)
#px4_logs = base_path + "px4_logs/2023-03-17/12_47_41.ulg"

# Summit

# prefix = base_path + "drone/radar_data/20230807-summit-day18-3start/20230807_064222" # flight 1 -- with payload SN01, suspected bad -- cannot see anything
# px4_logs = base_path + "px4_logs/2023-08-07/13_37_49.ulg"

# prefix = base_path + "drone/radar_data/20230807-summit-day18-3start/20230807_083256" # flight 2 -- no presums because of software version issue [NO PRESUMS, NEEDS PHASE INVERSION]
# px4_logs = base_path + "px4_logs/2023-08-07/15_27_05.ulg"
# need_postprocessing_phase_inversion = True

prefix = base_path + "drone/radar_data/20230807-summit-day18-3start/20230807_095229" # flight 3
px4_logs = base_path + "px4_logs/2023-08-07/16_46_01.ulg"
need_postprocessing_phase_inversion = True

# prefix = base_path + "drone/radar_data/20230807-summit-day18-3start/20230807_115113" # flight 4
# px4_logs = base_path + "px4_logs/2023-08-07/18_38_37.ulg"

# Summit Day 20
# prefix = base_path + "drone/radar_data/20230809-summit-day20-icesatappr/20230809_053300"
# px4_logs = base_path + "px4_logs/2023-08-09/12_25_17.ulg"



In [ ]:
# resave data as zarr for dask processing
zarr_base_location=base_path + "test_tmp_zarr_cache/"
zarr_path = pr.save_radar_data_to_zarr(prefix, zarr_base_location=zarr_base_location, skip_if_cached=True)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})
raw = pr.construct_slow_time_vectors(raw, allow_reading_data=True)

In [ ]:
if need_postprocessing_phase_inversion:
    phase_codes_path = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/sdr/phase_code.bin"
    raw= pr.invert_phase_dithering(raw, phase_codes_path)

In [ ]:
# Open the PX4 log file
px4_data = px4_log_loader.ulog_to_xarray(px4_logs,
                                         message_types_whitelist=[
                                             ("vehicle_gps_position", 0),
                                             ("distance_sensor", 0),
                                             ('vehicle_global_position', 0),
                                             ('vehicle_local_position', 0),
                                             ('vehicle_attitude', 0),
                                             ('wind', 0)
                                             ])

for key in px4_data.keys():
    px4_data[key] = px4_data[key].interp(timestamp_utc = raw.timestamp, method='nearest')

### Filter laser rangefinder data

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

gps_offset = 7 # m

px4_data[('distance_sensor', 0)]['current_distance'].plot.scatter(ax=ax, color='tab:blue', s=1, label='Laser Altimeter')
((-1 * px4_data[('vehicle_local_position', 0)]['z']) + gps_offset).plot.scatter(ax=ax, color='tab:orange', s=1, label='EKF2 Fused Local Altitude Estimate')

ax.legend()
ax.set_title('Altitude Estimates')

In [ ]:
# Select the best source for altitude estimation based on the plot above and local terrain
# If the surface is steeply sloped, you basically have to use the laser altimeter, as the EKF2
# altitude estimate knows nothing about the terrain.
# If the surface is flat, however, the GPS-derived (EKF2 estimate) is likely to be less noisy.

altitude_estimate_source = 'gps' # 'gps' or 'laser'

In [ ]:
if altitude_estimate_source == 'laser':
    distance_agl_filt = px4_data[('distance_sensor', 0)]['current_distance'].copy()

    # Remove anything with reported zero signal quality
    distance_agl_filt[px4_data[('distance_sensor', 0)]['signal_quality'] == 0] = np.nan
    # Remove anything where the reported distance is less than the minimum detectable range of 0.2 m
    distance_agl_filt[distance_agl_filt < 0.2] = np.nan

    for j in range(600):
        distance_agl_filt = distance_agl_filt.dropna(dim='pulse_idx')
        noisy_diff_mask = distance_agl_filt.diff(dim='pulse_idx') < -10
        distance_agl_filt[1:][noisy_diff_mask] = np.nan

    # Rolling average
    distance_agl_filt = distance_agl_filt.rolling(pulse_idx=400, center=True).median(skipna=True)
    distance_agl_filt = distance_agl_filt.dropna(dim='pulse_idx')
elif altitude_estimate_source == 'gps':
    distance_agl_filt = -1 * px4_data[('vehicle_local_position', 0)]['z'].copy()

    distance_agl_filt += gps_offset

    # Rolling average
    distance_agl_filt = distance_agl_filt.rolling(pulse_idx=20, center=True).median(skipna=True)
    distance_agl_filt = distance_agl_filt.dropna(dim='pulse_idx')


fig, ax = plt.subplots(figsize=(10, 5))
distance_agl_filt.plot(x='slow_time', marker=".", linewidth=1, ax=ax)
ax.set_ylabel("Filtered distance AGL [m]")
plt.show()

agl_uncertainty_m = 25 # Range over which to search for the reflecition peak

### Process Radar Data

In [ ]:
zero_sample_idx = 159 # B205mini, fs = 56 MHz

nstack = 100 # number of pulses to stack

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "blackman" # what you want to change the rx window to if modify_rx_window is true

# Note: Use a dielectric constant of 1 (free space) to get the surface reflection distance right
dielectric_constant = 1 # air
#dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
#dielectric_constant = 2.2957 # COAX (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

# Generate reference chirp

if modify_rx_window:
    config = copy.deepcopy(raw.config)
    config['GENERATE']['window'] = rx_window
else:
    config = raw.config

chirp_ts, ref_chirp = generate_chirp(config)

In [ ]:
def label_axis(ax, text_y=1.05):
    # Label plot with data source and processing
    ax.text(0, text_y, prefix.split("/")[-1] + "\n" + f"n_stack * num_presums = {nstack * raw.config['CHIRP']['num_presums']}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, fontdict={'size': 8})

prefix_tag = prefix.split("/")[-1] + "_TEST_"

In [ ]:
single_pulse_raw = raw.radar_data[{'pulse_idx': 10001}].compute()
plot1 = np.real(single_pulse_raw).hvplot.line(x='fast_time', color='red') * np.imag(single_pulse_raw).hvplot.line(x='fast_time')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

In [ ]:
(np.sum(np.abs(raw.radar_data) > 0.98) / np.product(raw.radar_data.shape)).compute()

In [ ]:
stacked = pr.remove_errors(raw) # Remove errors

stacked = pr.stack(stacked, nstack) # stack 

In [ ]:
compressed = pr.pulse_compress(stacked, ref_chirp,
                               fs=stacked.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed)

compressed_power = xr.apply_ufunc(
    lambda x: 20*np.log10(np.abs(x)),
    compressed,
    dask="parallelized"
)

In [ ]:
raw.config['GENERATE']['chirp_length'] * sig_speed/2

In [ ]:
compressed_power

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
mean_power_by_dist = compressed_power.where(compressed_power.pulse_idx < 5).where(np.isfinite(compressed_power.radar_data)).mean(dim='pulse_idx', skipna=True).radar_data
mean_power_by_dist.plot(x='reflection_distance', ax=ax)


peak_idxs, peak_properties = scipy.signal.find_peaks(mean_power_by_dist.to_numpy(), threshold=5)

print(peak_idxs)
print(mean_power_by_dist.reflection_distance[peak_idxs])

filter_half_width_m = 3
filter_half_width_zero_m = 30
#noise_floor_min_distance = 4000
#noise_floor_min_distance = 1000
noise_floor_min_distance = 400
#noise_floor_min_distance = 200

#ax.set_xlim(0, noise_floor_min_distance * 1.5)
#ax.set_xlim(0, 10000)

for pidx in peak_idxs:
    ax.axvspan(mean_power_by_dist.reflection_distance[pidx] - filter_half_width_m,
               mean_power_by_dist.reflection_distance[pidx] + filter_half_width_m, color='red', alpha=0.2)
    
ax.axvspan(0, filter_half_width_zero_m, color='gray', alpha=0.2)


# Estimate noise floor
noise_floor = mean_power_by_dist.where(mean_power_by_dist.reflection_distance > noise_floor_min_distance).mean(skipna=True)
noise_floor = noise_floor.to_numpy().item()
print(f"Noise floor: {noise_floor} dB (estimated from distances > {noise_floor_min_distance} m)")
ax.plot([noise_floor_min_distance, mean_power_by_dist.reflection_distance[-1]], [noise_floor, noise_floor], color='black')

label_axis(ax)
fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_sidelobe_filtering.png")

plt.show()

In [ ]:
distance_agl_filt_interp = distance_agl_filt.interp(pulse_idx=compressed_power.pulse_idx, method='nearest')
dist_from_surf_est = compressed_power.reflection_distance - distance_agl_filt_interp

# Mask around where the surface reflection could plausibly be based on the laser data
compressed_power_masked = compressed_power.where(np.abs(dist_from_surf_est) < agl_uncertainty_m)

# Mask out the main direct path
compressed_power_masked = compressed_power_masked.where(compressed_power_masked.reflection_distance > filter_half_width_zero_m)

# Mask out anywhere the surface will fall too close to the direct path
compressed_power_masked = compressed_power_masked[{'pulse_idx': distance_agl_filt_interp > filter_half_width_zero_m + 10}]

# Mask out the peaks that might be due to sidelobes
for pidx in peak_idxs:
    compressed_power_masked = compressed_power_masked.where(
        np.abs(compressed_power_masked.reflection_distance - mean_power_by_dist.reflection_distance[pidx]) > filter_half_width_m)

# Swap reflection_distance to be the primary coordinate instead of travel_time
compressed_power_masked = compressed_power_masked.swap_dims({'travel_time': 'reflection_distance'})

picked_surface = compressed_power_masked.radar_data.idxmax(dim='reflection_distance', skipna=True, fill_value=np.nan)

In [ ]:
cmp_pwr_masked_np = compressed_power_masked.radar_data.transpose().to_numpy()
picked_surface = picked_surface.compute()


In [ ]:
# fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')
# p = ax.pcolormesh(compressed_power_masked.timestamp, compressed_power_masked.reflection_distance, cmp_pwr_masked_np, shading='auto', cmap='inferno')
# ax.invert_yaxis()
# clb = fig.colorbar(p, ax=ax)
# clb.set_label('Return Power [dB]')
# ax.set_xlabel('Slow Time [s]')
# ax.set_ylabel('Distance to Reflector [m]')
# # relevant options: ax.set_ylim=(100,-50), ax.set_xlim=(0, 1), vmin=-90, vmax=40
# ax.set_ylim(100, -50)

# picked_surface.plot(ax=ax, x='timestamp', marker='.', linewidth=0, alpha=0.5)

# label_axis(ax)
# fig.tight_layout()
# fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_filtered_picks.png")

# plt.show()

In [ ]:
# Plot radargram
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')

p = ax.pcolormesh(compressed_power.timestamp, compressed_power.reflection_distance, compressed_power.radar_data.transpose(), shading='auto', cmap='inferno')
ax.invert_yaxis()
clb = fig.colorbar(p, ax=ax)
clb.set_label('Return Power [dB]')
ax.set_xlabel('Slow Time [s]')
ax.set_ylabel('Distance to Reflector [m]')
# relevant options: ax.set_ylim=(100,-50), ax.set_xlim=(0, 1), vmin=-90, vmax=40
ax.set_ylim(200, -50)
#ax.set_ylim(2000, 0)

# Plot expected surface reflection distance

ax.plot(distance_agl_filt_interp.timestamp, distance_agl_filt_interp, color='white', linewidth=2, linestyle=":")
ax.fill_between(distance_agl_filt_interp.timestamp, distance_agl_filt_interp - agl_uncertainty_m, distance_agl_filt_interp + agl_uncertainty_m, color='white', alpha=0.2)

picked_surface.plot(ax=ax, x='timestamp', marker='.', linewidth=0, alpha=0.5)

label_axis(ax)
fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_picks.png")

plt.show()

In [ ]:
def quaternion_to_euler(q0, q1, q2, q3):
    """
    Convert quaternion (q0, q1, q2, q3) to Euler angles (roll, pitch, yaw).
    """
    # Roll (x-axis rotation)
    sinr_cosp = 2 * (q0 * q1 + q2 * q3)
    cosr_cosp = 1 - 2 * (q1 * q1 + q2 * q2)
    roll = np.arctan2(sinr_cosp, cosr_cosp)

    # Pitch (y-axis rotation)
    sinp = 2 * (q0 * q2 - q3 * q1)
    pitch = np.arcsin(sinp)
    #pitch = np.where(np.abs(sinp) >= 1, np.sign(sinp) * np.pi / 2, np.arcsin(sinp))

    # Yaw (z-axis rotation)
    siny_cosp = 2 * (q0 * q3 + q1 * q2)
    cosy_cosp = 1 - 2 * (q2 * q2 + q3 * q3)
    yaw = np.arctan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

# Apply the function to the dataset
q0 = px4_data[('vehicle_attitude', 0)]['q[0]']
q1 = px4_data[('vehicle_attitude', 0)]['q[1]']
q2 = px4_data[('vehicle_attitude', 0)]['q[2]']
q3 = px4_data[('vehicle_attitude', 0)]['q[3]']

roll, pitch, yaw = quaternion_to_euler(q0, q1, q2, q3)

px4_data[('vehicle_attitude', 0)]['roll'] = roll
px4_data[('vehicle_attitude', 0)]['pitch'] = pitch
px4_data[('vehicle_attitude', 0)]['yaw'] = yaw

In [ ]:
# Pick the return power along the travel time index

tt_dropped = picked_surface.dropna(dim='pulse_idx')

fig, axs = plt.subplots(6,1, figsize=(10, 12), sharex=True)
# swap the primary coordinate to refleciton distance
compressed_power_tmp = compressed_power.swap_dims({'travel_time': 'reflection_distance'})
surface_power = compressed_power_tmp.radar_data.sel(reflection_distance=tt_dropped, pulse_idx=tt_dropped.pulse_idx, method='nearest')
surface_power = surface_power.persist()
surface_power.plot(ax=axs[0], x='timestamp', marker='.', linewidth=0, alpha=0.5, label='Surface reflection power')
axs[0].set_ylabel('Surface Reflection Power [dB]')

ax.axhline(noise_floor, color='black', linestyle='--', label='Noise floor')

# Plot the distance to the surface on axs[1]
distance_agl_filt_interp.plot(ax=axs[1], x='timestamp', marker='.', linewidth=0, alpha=0.5, label='Distance AGL')
# Plot the roll, pitch, and yaw
px4_data[('vehicle_attitude', 0)]['roll'].plot(ax=axs[2], x='timestamp_utc', marker='.', linewidth=0, alpha=0.5, label='Roll angle')
px4_data[('vehicle_attitude', 0)]['pitch'].plot(ax=axs[3], x='timestamp_utc', marker='.', linewidth=0, alpha=0.5, label='Pitch angle')
px4_data[('vehicle_attitude', 0)]['yaw'].plot(ax=axs[4], x='timestamp_utc', marker='.', linewidth=0, alpha=0.5, label='Yaw angle')
np.sqrt(px4_data[('wind', 0)]['windspeed_north']**2 + px4_data[('wind', 0)]['windspeed_east']**2).plot(ax=axs[5], x='timestamp_utc', marker='.', linewidth=0, alpha=0.5)
axs[5].set_ylabel('Wind Speed [m/s]')

for ax in axs:
    ax.grid(True)

label_axis(axs[0])
fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_surface_reflection.png")

plt.show()

In [ ]:
# Merge distance_agl_filt_interp with surface_power by timestamp

surface_power_ds = surface_power.to_dataset(name='surface_power').copy()
surface_power_ds['distance_agl'] = distance_agl_filt_interp.interp(pulse_idx=surface_power.pulse_idx, method='nearest')
surface_power_ds['roll'] = px4_data[('vehicle_attitude', 0)]['roll'].interp(pulse_idx=surface_power.pulse_idx, method='nearest')
surface_power_ds['abs_roll'] = np.abs(surface_power_ds['roll'])
surface_power_ds['pitch'] = px4_data[('vehicle_attitude', 0)]['pitch'].interp(pulse_idx=surface_power.pulse_idx, method='nearest')
surface_power_ds['abs_pitch'] = np.abs(surface_power_ds['pitch'])
surface_power_ds = surface_power_ds.dropna(dim='pulse_idx').persist()

surface_power_ds.plot.scatter(x='abs_pitch', y='surface_power', label='Surface reflection power', c="tab:blue", linewidths=0, s=3)

In [ ]:
surface_power_ds.plot.scatter(x='abs_roll', y='surface_power', label='Surface reflection power', c="tab:blue", linewidths=0, s=3)

In [ ]:
geometric_spreading_correction = 10 * np.log10(surface_power_ds['reflection_distance']**2)
surface_power_ds['surface_power_corrected'] = surface_power_ds['surface_power'] + geometric_spreading_correction
surface_power_ds['surface_power_corrected'] = surface_power_ds['surface_power_corrected'].where(np.isfinite(surface_power_ds['surface_power_corrected']))

fig, axs = plt.subplots(4,1, figsize=(10, 6), sharex=True)

surface_power_ds.plot.scatter(ax=axs[0], x='timestamp', y='surface_power', color='red', label='Surface reflection power', s=1)
surface_power_ds.plot.scatter(ax=axs[1], x='timestamp', y='surface_power_corrected', color='red', label='Surface reflection power (corrected)', s=1)

surface_power_ds.plot.scatter(ax=axs[2], x='timestamp', y='abs_roll', color='blue', label='Roll angle', s=1)

surface_power_ds_filt = surface_power_ds.where(surface_power_ds['abs_roll'] < 5*(np.pi/180)).dropna(dim='pulse_idx')

surface_power_ds_filt.plot.scatter(ax=axs[3], x='timestamp', y='surface_power_corrected', color='red', s=1, label='Surface reflection power')

surface_power_corrected_mean = surface_power_ds_filt['surface_power_corrected'].mean(skipna=True).to_numpy().item()
surf_power_corrected_lin = 10**(surface_power_ds_filt['surface_power_corrected']/10)
surf_power_corrected_std_lin = surf_power_corrected_lin.std(skipna=True).to_numpy().item()
surface_power_corrected_std = 10*np.log10(surf_power_corrected_std_lin)
print(f"Mean surface reflection power w/ geometric spreading correction: {surface_power_corrected_mean} dB")
print(f"Std dev surface reflection power w/ geometric spreading correction: {surface_power_corrected_std} dB")

axs[3].axhline(surface_power_corrected_mean, color='black', linestyle='--', label='Mean power')
axs[3].axhspan(surface_power_corrected_mean - surface_power_corrected_std, surface_power_corrected_mean + surface_power_corrected_std, color='gray', alpha=0.2, label='Std dev')

for ax in axs:
    ax.grid(True)

# ymin, ymax = axs[0].get_ylim()
# axs[1].set_ylim(ymin, 0)
# axs[3].set_ylim(ymin, 0)

label_axis(ax)
fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_roll_filter.png")

In [ ]:
# Plot estimated surface return power as a funciton of altitude

alt_m = np.arange(10, 120, 10)
surface_power_est = surface_power_corrected_mean + 10 * np.log10(1/(alt_m**2))
surface_snr_est = surface_power_est - noise_floor

fig, (ax_surf, ax_snr) = plt.subplots(2,1, figsize=(6, 6), sharex=True)
ax_surf.plot(alt_m, surface_power_est)
ax_surf.fill_between(alt_m, surface_power_est - surface_power_corrected_std, surface_power_est + surface_power_corrected_std, color='gray', alpha=0.2)
ax_surf.set_ylabel('Estimated surface return\npower [dB]')

surface_power_ds_filt.plot.scatter(x='reflection_distance', y='surface_power', color='red', label='Surface reflection power', s=1, alpha=0.5, ax=ax_surf)

ax_snr.plot(alt_m, surface_snr_est)
ax_snr.fill_between(alt_m, surface_snr_est - surface_power_corrected_std, surface_snr_est + surface_power_corrected_std, color='gray', alpha=0.2)
ax_snr.set_ylabel('Estimated surface SNR [dB]')
ax_snr.set_xlabel('Altitude [m]')

for ax in (ax_surf, ax_snr):
    ax.grid(True)
    ax.set_xlim(20,110)

label_axis(ax_surf)
ax_surf.text(1, 1.05,
             f"Surface SNR Corrected: {surface_power_corrected_mean:.2f} dB\nNoise floor: {noise_floor:.2f} dB",
             horizontalalignment='right', verticalalignment='center',
             transform=ax_surf.transAxes, fontdict={'size': 8})

fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_surface_snr.png")
plt.show()

In [ ]:
# Plot estimated surface return power as a funciton of altitude

alt_m = np.arange(10, 120, 10)
surface_power_est = surface_power_corrected_mean + 10 * np.log10(1/(alt_m**2))

fig, ax_surf = plt.subplots(1,1, figsize=(6, 4))
fitline = ax_surf.plot(alt_m, surface_power_est, color='tab:blue', linestyle='--')
ax_surf.set_ylabel('Estimated surface return\npower [dB]')

scat = surface_power_ds_filt.plot.scatter(x='reflection_distance', y='surface_power', color='tab:orange', label='Surface reflection power', s=1, alpha=0.2, ax=ax_surf)

ax_surf.grid(True)
ax_surf.set_xlim(30,110)

label_axis(ax_surf, text_y=1.06)
ax_surf.text(1, 1.06,
             f"Surface SNR Corrected: {surface_power_corrected_mean:.2f} dB\nNoise floor: {noise_floor:.2f} dB",
             horizontalalignment='right', verticalalignment='center',
             transform=ax_surf.transAxes, fontdict={'size': 8})

ax_surf.set_xlabel('Altitude above ground level (AGL) [m]')
ax_surf.set_ylabel('Surface return power [dB]')

legend1 = ax_surf.legend([scat, fitline[0]], ['Picked surface reflection power', 'Expected geometric spreading'], loc='upper right')

# For readability, reset opacity for the legend
for lh in legend1.legend_handles:
    if isinstance(lh, matplotlib.collections.PathCollection):
        lh.set_alpha(1)

fig.tight_layout()
fig.savefig(f"snr_outputs/{prefix_tag}_nstack{nstack}_surface_power_vs_altitude.png")

In [ ]:
# Save a pickle file with the corrected surface power, noise floor, and raw.config
import pickle
import datetime

rx_window_modified = None
if modify_rx_window:
    rx_window_modified = rx_window

with open(f"snr_outputs/{prefix_tag}_nstack{nstack}_results.pickle", "wb") as f:
    pickle.dump({"surface_power_corrected_mean": surface_power_corrected_mean,
                 "surface_power_corrected_std": surface_power_corrected_std,
                 "noise_floor": noise_floor,
                 "config": raw.config,
                 "nstack": nstack,
                 "rx_window_modified": rx_window_modified,
                 "run_timestamp": datetime.datetime.now(),
                 "prefix": prefix_tag,
                 "surface_power_dataset": surface_power_ds_filt
                 }, f)

In [ ]:
# import statsmodels.api as sm

# # Prepare the data
# X = surface_power_ds.to_pandas()[['distance_agl', 'abs_roll', 'abs_pitch']]
# y = surface_power_ds.to_pandas()['surface_power']

# # Add a constant to the independent variables matrix
# X = sm.add_constant(X)

# # Fit the linear regression model
# model = sm.OLS(y, X).fit()

# # Print the coefficients
# print(model.summary())


In [ ]:
print(datetime.datetime.now() - t_start)

In [ ]:
notebook_filename = f"snr_outputs/{prefix_tag}_nstack{nstack}_notebook.ipynb"
%notebook $notebook_filename